In [ ]:
%pip install pandas
%pip install langchain_community

In [1]:
import pandas as pd
from langchain_community.llms import Ollama

In [20]:
llm = Ollama(model="risk_analizer_3.1")
llm.invoke("Pérdida, hurto o deterioro de un activo por un acceso no autorizado al área del Archivo")

'Riesgo operativo.'

In [22]:
df = pd.read_csv('data.csv')
df

,Riesgo,Clasificación
0,Ausencia de personal suficiente para atender o...,NaN
1,Remisión de solicitudes de información de clie...,NaN
2,Eventual exceso del ejercicio de la competenci...,NaN
3,Probabilidad de aprobación de presupuestos qu...,NaN
4,Posibilidad de disminución en la calidad del p...,NaN
...,...,...
624,Operación fraudulenta por parte funcionarios q...,NaN
625,Pago de facturas que no cumplen con las condic...,NaN
626,Pagos sin contenido presupuestario parcial o t...,NaN
627,Reintegros al FFCC con montos incorrectos,NaN


In [23]:
def categorizar_riesgo(risk, model="risk_analizer_llama3",contador=0, palabrasMaximas=2, largoMaximo=30):
    llm = Ollama(model=model)
    response = llm.invoke(risk).split("\n")[0]
    
    if contador > 3:
        return "No se pudo generar una respuesta" 

    if len(response.split(" ")) > palabrasMaximas:
        print(f"Respuesta mayor a {palabrasMaximas} palabras: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)
    
    if len(response) > largoMaximo:
        print(f"Respuesta muy larga: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)
    
    valoresMalos = ["alto", "bajo", "falso"]
    if any(valor in response.lower() for valor in valoresMalos):
        print(f"Respuesta tiene valores que no coinciden: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)

    # Limpieza de caracteres no deseados
    response = response.replace("*", "").replace(".", "").replace(":", "").replace('"', "").title()
    
    print(f"Respuesta normal: {response}")
    return response
    


In [98]:
# Leer archivo
df = pd.read_csv('data_clasificada_v2.csv')
# recorrer la columna de riesgos y categorizar el riesgo
for i in range(len(df)):
    
    # Categorizar el riesgo
    resultado = categorizar_riesgo(df["Riesgo"][i], model="risk_analizer_description_llama3", palabrasMaximas=15, largoMaximo=50)
    
    #Guardar la clasificacion en una nueva columna
    df.loc[i, "Descripción"] = resultado
    
# Imprimir el dataframe con la nueva columna
df

# Guardar los resultados en un nuevo archivo
df.to_csv('data_clasificada_v3.csv', index=False)

Respuesta normal: Falta De Recursos Humanos
Respuesta normal: Demora En Respuesta A Consultas
Respuesta normal: Competencia Por Recursos Públicos
Respuesta normal: Presupuesto Irregular
Respuesta normal: Calidad Productiva En Peligro
Respuesta normal: Desacuerdo En La Asignación De Fondos
Respuesta normal: Riesgo De Datos Erróneos Y Fallas Técnicas
Respuesta normal: Confianza Ciega En Datos Inverificables
Respuesta normal: Falta De Autoridad
Respuesta normal: Pérdida De Expertise Crítica
Respuesta normal: Control De Calidad Ineficaz
Respuesta normal: Conflictos De Interés Y Fraude Presupuestario
Respuesta normal: Rechazo Presupuesto Por Fallo Tecnológico
Respuesta normal: Errores En Presupuestos
Respuesta normal: Presupuesto Agotado En Fin De Año
Respuesta normal: Facturas No Ajustadas Al Pedido
Respuesta normal: Error De Pago Incorrecto
Respuesta normal: Fraude En Pagos A Proveedores
Respuesta normal: Fraude En Pagos A Proveedores
Respuesta normal: Perdida De Fondos Personales
Respues

In [27]:
# Generar un archivo solo con las clasificaciones
df_clasificacion = df["Clasificación"]
df_clasificacion.to_csv('clasificacion.csv', index=False)

# Leer el archivo con las clasificaciones y agregarlo a un string
clasificacion = ""
with open('clasificacion.csv', 'r') as file:
    clasificacion = file.read()
    
clasificacion

# Enviarlo al modelo para que lo clasifique
llm = Ollama(model="llama3")
response = llm.invoke("Analizar todas las categorias de riesgo y crear 10 categorias generales en las que se puedan clasificar cada una de esas subclasificaciones " + clasificacion)
response = response.split("\n")
response


['Después de analizar las categorías de riesgo, he identificado 10 categorías generales que abarcan la mayoría de los riesgos:',
 '',
 '1. **Riesgo administrativo**: Riesgo relacionado con la gestión y toma de decisiones dentro de una organización.',
 '2. **Riesgo ético**: Riesgo que se refiere a la moralidad, integridad y valores en la toma de decisiones.',
 '3. **Riesgo tecnológico**: Riesgo relacionado con la tecnología, incluyendo la seguridad de la información y el uso no autorizado de datos.',
 '4. **Riesgo operativo**: Riesgo que se refiere a la capacidad de una organización para realizar sus actividades diarias sin interrupciones.',
 '5. **Riesgo financiero**: Riesgo relacionado con la gestión del dinero y los recursos financieros.',
 '6. **Riesgo crítico**: Riesgo extremadamente alto que puede tener un impacto significativo en la organización o sus stakeholders.',
 '7. **Riesgo de seguridad**: Riesgo que se refiere a la protección de personas, propiedad y datos contra daños o 

In [30]:
# Crear una categoria macro para cada categoria
df = pd.read_csv('data_clasificada.csv')
# recorrer la columna de riesgos y categorizar el riesgo
for i in range(len(df)):
    
    # Categorización
    resultado = categorizar_riesgo(df["Clasificación"][i], model="risk_analizer_macro_llama3")
    
    #Guardar la clasificacion en una nueva columna
    df.loc[i, "Clasificación_Macro"] = resultado
    
# Imprimir el dataframe con la nueva columna
df

# Guardar los resultados en un nuevo archivo
df.to_csv('data_final.csv', index=False)